In [6]:
import numpy as np
import random

with open("cities.txt", 'r') as file:
    cities = [line.split() for line in file.read().splitlines()]

# Convert city coordinates to integers
cities = [(int(city[1]), int(city[2])) for city in cities]
num_cities = len(cities)

# Create a distance matrix
def calculate_distance(city1, city2):
    return np.linalg.norm(np.array(city1) - np.array(city2))

distance_matrix = np.zeros((num_cities, num_cities))
for i in range(num_cities):
    for j in range(num_cities):
        distance_matrix[i][j] = calculate_distance(cities[i], cities[j])

# Rest of the code remains the same as before...
# (Genetic Algorithm Parameters, Functions, Main Loop)

# Genetic Algorithm Parameters
population_size = 50
num_generations = 100
mutation_rate = 0.01

# Initialize a population randomly
def create_individual():
    return np.random.permutation(num_cities)

# ... (Remaining code for genetic algorithm)
population = [create_individual() for _ in range(population_size)]

# Fitness function
def calculate_fitness(individual):
    total_distance = sum(distance_matrix[individual[i % num_cities]][individual[(i + 1) % num_cities]] for i in range(num_cities))
    return 1 / total_distance

# Selection function (Tournament selection)
def select_parents(population, fitness_values):
    parent1 = random.choice(population)
    parent2 = random.choice(population)
    if fitness_values[parent1] > fitness_values[parent2]:
        return parent1
    else:
        return parent2

# Crossover function (Partially Mapped Crossover - PMX)
def crossover(parent1, parent2):
    start = random.randint(0, len(parent1) - 1)
    end = random.randint(start + 1, len(parent1))

    child = [-1] * len(parent1)
    child[start:end] = parent1[start:end]

    parent1_list = parent1.tolist()
    parent2_list = parent2.tolist()

    for i in range(start, end):
        if parent2[i] not in child:
            idx = parent2_list.index(parent1[i])
            while child[idx] != -1:
                idx = parent2_list.index(parent1[idx])
            child[idx] = parent2[i]

    for i in range(len(child)):
        if child[i] == -1:
            child[i] = parent2[i]

    return np.array(child)


# Mutation function (Swap mutation)
def mutate(individual):
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(num_cities), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual
# Main Genetic Algorithm loop
fitness_values = []

for _ in range(num_generations):
    fitness_values = [(individual, calculate_fitness(individual)) for individual in population]
    fitness_values.sort(key=lambda x: x[1], reverse=True)

    new_population = []

    for _ in range(population_size // 2):
        parent1, _ = random.choice(fitness_values[:population_size // 2])
        parent2, _ = random.choice(fitness_values[:population_size // 2])
        
        child1 = crossover(parent1, parent2)
        child1 = mutate(child1)
        
        child2 = crossover(parent2, parent1)
        child2 = mutate(child2)

        new_population.extend([child1, child2])

    population = new_population

# Find the best route in the final population
best_route, best_fitness = max(fitness_values, key=lambda x: x[1])

print("Best route:", best_route)
print("Best distance:", 1 / best_fitness)


Best route: [ 40  19  12  15  42  49  37  47  77  76  68  92  78  69  66  71  67  61
  32  21  22  63  75  73  25   4  17   2  70  58  83   3  36  23  98  97
  31  50  38  48  43  56  72  74  65  94  79  55   5   7  10   0  20  35
  30  86  96  64  60  91  93 105  57  95  62  87  59 102 103 106  54  52
  45  24  33  90  88  99 100 104  80  26   8  44  46  39   6  84  85  82
  81 101   1  27  29  28  41  89  53  51  14  11  18   9  13  16  34]
Best distance: 300306.0054824662
